In [ ]:
!pip install kaggle
!pip install opencv-python
from kaggle.api.kaggle_api_extended import KaggleApi

import json
import os
from datetime import datetime
from IPython.display import display, HTML
from IPython.display import display
from IPython.display import Image as IPyImage, display
!pip install opencv-python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import random
from PIL import Image,ImageDraw, ImageFont
import zipfile
import shutil
import math
from skimage.color import rgb2gray
from skimage import exposure
from skimage.filters import median
from skimage.morphology import disk,opening, closing
from skimage.filters import threshold_otsu, try_all_threshold
from skimage.measure import shannon_entropy,regionprops, label 
from skimage.feature import graycomatrix, graycoprops
from skimage.color import label2rgb
from scipy.stats import skew, kurtosis,entropy
from collections import Counter










jsnApi = '{"username":"[merkarada]","key":"[KGAT_e706c4081d47addd99d972a4f0741f9c]"}'
home_dir = os.path.expanduser('~') 
kaggle_dir = os.path.join(home_dir, 'scin.kaggle') 
kaggle_json_path = os.path.join(kaggle_dir, '.json')

if not os.path.exists(kaggle_dir):
    print(f"'.kaggle' klasörü oluşturuluyor: {kaggle_dir}")
    os.makedirs(kaggle_dir, exist_ok=True)

try:
    with open(kaggle_json_path, 'w', encoding='utf-8') as f:
        f.write(jsnApi)
    
  
    os.chmod(kaggle_json_path, 0o600)

    display(HTML("<b>Kaggle API Anahtarı Başarıyla Tanımlandı.</b>"))
    display(HTML("<b>Projede kullanılacak olan kütüphaneler yüklendi.</b>"))
    display(HTML("<b>İşlemlere devam edebilirsiniz.</b>"))
    display(HTML("<b>---------------------------------------------------------------------------------------</b>"))
    
except Exception as e:
    print(f" HATA: Dosya yazma işlemi başarısız oldu. Hata: {e}")

#### Veri Setinin Kaggle Platformundan indirilmesi (Skin-Cancer9-lassesisic Zip Dosyası)

In [ ]:
display(HTML("<b>Kaggle'dan indirme başlatılıyor....</b>"))
api = KaggleApi()
api.authenticate()
slugDs = 'nodoubttome/skin-cancer9-classesisic'
api.dataset_download_files(slugDs, path='.', unzip=False) 
fileWithZip = f"{slugDs.split('/')[-1]}.zip" 
print(f"İndirme tamamlandı:{fileWithZip}")

#### Skin-Cancer9-lassesisic Zip Dosyası Dataların Çıkarılması

In [ ]:
print(f"'{fileWithZip}' dosyası açılıyor...")

try:
    with zipfile.ZipFile(fileWithZip, 'r') as opnZip:        
        opnZip.extractall('.')    
    print("Dosya başarıyla açıldı.")    
    dataPath = os.path.join(os.getcwd(), 'Train')    
    print(f"{dataPath}")

except FileNotFoundError:
    print(f"HATA: {fileWithZip} dosyası bulunamadı. Lütfen indirme adımının çalıştığından emin olun.")
except Exception as e:
    print(f"HATA: Zip dosyası açılırken sorun oluştu: {e}")

#### Çalışılacak Görsellerin Rastgele Seçilmesi

In [ ]:
imgPath = os.getcwd() 
dataPath = os.path.join(imgPath, 'Train')

all_image_paths = []

#print(f"Başlangıç Klasörü {imgPath}")

try:
    
    for root, dirs, files in os.walk(imgPath):
        for file in files:            
            if file.lower().endswith(('.jpg', '.png', '.jpeg')):               
                full_path = os.path.join(root, file)
                all_image_paths.append(full_path)

except Exception as e:
    print(f"Dosya okuma sırasında beklenmeyen hata oluştu: {e}") 
train_df = pd.DataFrame({'image_path': all_image_paths})
total_images = len(train_df)

## print(f"\nToplam Görüntü Sayısı: {total_images}")


NUM_IMAGES = 20         # Görsel Sayısı
BASE_DIR = os.getcwd()
WORK_DIR = "work_dir"

run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
TARGET_DIR = os.path.join(WORK_DIR, f"selected_images_{run_id}")
SCREENSHOT_PATH = os.path.join(TARGET_DIR, "selected_image_paths.png")

os.makedirs(TARGET_DIR, exist_ok=True)

if NUM_IMAGES > len(all_image_paths):
    raise ValueError("İstenen sayı toplam görselden fazla!")

selected_paths = random.sample(all_image_paths, NUM_IMAGES)
final_paths = []

for src in selected_paths:
    filename = os.path.basename(src)
    dst = os.path.join(TARGET_DIR, filename)
    shutil.copy(src, dst)
    final_paths.append(dst)




img_width = 1600
line_height = 25
img_height = max(400, line_height * (len(final_paths) + 2))

img = Image.new("RGB", (img_width, img_height), "white")
draw = ImageDraw.Draw(img)

try:
    font = ImageFont.truetype("arial.ttf", 18)
except:
    font = ImageFont.load_default()

y = 20
for line in final_paths:
    draw.text((20, y), line, fill="black", font=font)
    y += line_height

img.save(SCREENSHOT_PATH)

display(IPyImage(filename=SCREENSHOT_PATH))

print(f"Toplam Skin cancer ISIC The International Skin Imaging Collaboration dataseti {total_images} adettir.\nBu datasetin içerisinden çalışmak için {NUM_IMAGES} adet görsel rastgele seçilmiştir.")
print(f"Klasör: {TARGET_DIR}")

#### Seçilen Görsellerin Orjinal Gösterimi

In [ ]:

IMG_SIZE = (150, 150)
COLS = 5
rows = math.ceil(len(final_paths) / COLS)

plt.figure(figsize=(COLS * 3, rows * 3))

for i, img_path in enumerate(final_paths):
    filename = os.path.basename(img_path)
    img = Image.open(img_path)
    img.thumbnail(IMG_SIZE)

    ax = plt.subplot(rows, COLS, i + 1)
    ax.imshow(img)
    ax.axis("off")

  
    ax.set_title(str(i + 1), fontsize=10)

   
    ax.text(
        0.5, -0.05, filename,
        ha="center",
        va="top",
        transform=ax.transAxes,
        fontsize=10,
        wrap=True
    )

plt.tight_layout()
plt.show()

print("")
print("Seçilen görseller ayrı bir klasör oluşturularak yeni klasöre taşınmıştır.\nDosya isimleri ve sıra numaraları ile görseller görüntülenmektedir.")
print("")

#### 1) RGB → Grayscale Dönüşümü 


In [ ]:
NUM_SHOW = NUM_IMAGES
NUM_SHOW = min(NUM_SHOW, len(final_paths))

plt.figure(figsize=(10, NUM_SHOW * 3))

for i in range(NUM_SHOW): 
    filename = os.path.basename(final_paths[i])   
    rgb_img = Image.open(final_paths[i]).convert("RGB")
    rgb_np = np.array(rgb_img) 
    gray_img = rgb2gray(rgb_np)
    plt.subplot(NUM_SHOW, 2, 2*i + 1)
    plt.imshow(rgb_np)
    plt.axis("off")
    plt.title(f"RGB\n{filename}", fontsize=9)
    plt.subplot(NUM_SHOW, 2, 2*i + 2)
    plt.imshow(gray_img, cmap="gray")
    plt.axis("off")
    plt.title(f"Grayscale\n{filename}", fontsize=9)

plt.tight_layout()
plt.show()
print("")
print("1) RGB → Grayscale: Renk bilgisi tek kanala indirgenerek parlaklık temelli analiz için uygun hale getirildi.")
print("")

### 2) Pre-Processing 

#### 2.1 Crop

In [ ]:

def crop_image_percent(img, crop_percent=0.20):
    """
    Görüntüyü kenarlardan sabit yüzde ile kırpar
    ROI kaybını önlemek için merkezi korur
    """
    w, h = img.size

    left   = int(w * crop_percent)
    right  = int(w * (1 - crop_percent))
    top    = int(h * crop_percent)
    bottom = int(h * (1 - crop_percent))

    return img.crop((left, top, right, bottom))

NUM_SHOW = NUM_IMAGES
NUM_SHOW = min(NUM_SHOW, len(final_paths))

plt.figure(figsize=(8, NUM_SHOW * 2.5))

for i in range(NUM_SHOW):
    img_path = final_paths[i]
    filename = os.path.basename(img_path)

    # Orijinal (RGB veya grayscale fark etmez)
    img = Image.open(img_path).convert("RGB")

    # Crop edilmiş görüntü
    cropped_img = crop_image_percent(img, crop_percent=0.10)

    # Orijinal
    plt.subplot(NUM_SHOW, 2, 2*i + 1)
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"Orijinal\n{filename}", fontsize=9)

    # Crop edilmiş
    plt.subplot(NUM_SHOW, 2, 2*i + 2)
    plt.imshow(cropped_img)
    plt.axis("off")
    plt.title("Crop Edilmiş", fontsize=9)

plt.tight_layout()
plt.show()

print("")
print("2.1 Crop: İlgi alanını (lezyon) koruyacak şekilde kenarlardan kırpma yapılarak arka plan ve çerçeve gürültüsü azaltıldı.")
print("")


#### 2.2 Kontrast İyileştirme 

In [ ]:

def crop_image_percent(img, crop_percent=0.20):
    w, h = img.size
    left   = int(w * crop_percent)
    right  = int(w * (1 - crop_percent))
    top    = int(h * crop_percent)
    bottom = int(h * (1 - crop_percent))
    return img.crop((left, top, right, bottom))
def histogram_equalization(gray_img):
    gray_np = np.array(gray_img)
    eq_img = exposure.equalize_hist(gray_np)
    return eq_img
    NUM_SHOW = NUM_IMAGES
NUM_SHOW = min(NUM_SHOW, len(final_paths))

plt.figure(figsize=(8, NUM_SHOW * 2.5))

for i in range(NUM_SHOW):
    img_path = final_paths[i]
    filename = os.path.basename(img_path)

   
    img = Image.open(img_path).convert("L")

    # Crop
    cropped_img = crop_image_percent(img, crop_percent=0.20)

    # Histogram Equalization
    eq_img = histogram_equalization(cropped_img)

    # Crop edilmiş
    plt.subplot(NUM_SHOW, 2, 2*i + 1)
    plt.imshow(cropped_img, cmap="gray")
    plt.axis("off")
    plt.title(f"Crop Edilmiş\n{filename}", fontsize=9)

    # Equalized
    plt.subplot(NUM_SHOW, 2, 2*i + 2)
    plt.imshow(eq_img, cmap="gray")
    plt.axis("off")
    plt.title("Histogram Eşitleme", fontsize=9)

plt.tight_layout()
plt.show()

print("")
print("2.2 Kontrast İyileştirme: Lezyon ve arka plan arasındaki ayrımı güçlendirmek amacıyla kontrast artırma işlemi uygulandı.")
print("")


#### 2.3 Gürültü Azaltma / Blurring 

In [ ]:
NUM_SHOW = NUM_IMAGES
NUM_SHOW = min(NUM_SHOW, len(final_paths))

plt.figure(figsize=(8, NUM_SHOW * 2.5))

for i in range(NUM_SHOW):
    img_path = final_paths[i]
    filename = os.path.basename(img_path)

    # Orijinal → Grayscale
    img = Image.open(img_path).convert("L")

    # Crop
    cropped_img = crop_image_percent(img, crop_percent=0.20)
    cropped_np = np.array(cropped_img)

    # Histogram Equalization
    equalized = exposure.equalize_hist(cropped_np)

    # Median Blur
    median_blurred = median(equalized, disk(3))

    # Equalized
    plt.subplot(NUM_SHOW, 2, 2*i + 1)
    plt.imshow(equalized, cmap="gray")
    plt.axis("off")
    plt.title("Equalized", fontsize=9)

    # Median Blur
    plt.subplot(NUM_SHOW, 2, 2*i + 2)
    plt.imshow(median_blurred, cmap="gray")
    plt.axis("off")
    plt.title("Median Blur", fontsize=9)

plt.tight_layout()
plt.show()
print("")
print("2.3 Gürültü Azaltma: Lezyon sınırlarını mümkün olduğunca koruyarak yüksek frekanslı gürültüyü azaltmak için blur işlemi uygulandı.")
print("")


### 3) Thresholding ile Segmentasyon 

#### 3.1 Eşik Değerlerinin Belirlenmesi

In [ ]:
sample_path = final_paths[0]
img = Image.open(sample_path).convert("L")

# Crop
cropped = crop_image_percent(img, crop_percent=0.10)
cropped_np = np.array(cropped)

# Histogram Equalization
equalized = exposure.equalize_hist(cropped_np)

# Median Blur
preprocessed = median(equalized, disk(3))



global_thresh = 0.5
binary_global = preprocessed > global_thresh

print("Global Threshold Değeri:", global_thresh)


# otsu

otsu_thresh = threshold_otsu(preprocessed)
binary_otsu = preprocessed > otsu_thresh

print("Otsu Threshold Değeri:", otsu_thresh)

# Karşılaştırmalı Threshold

fig, ax = try_all_threshold(preprocessed, figsize=(10, 8), verbose=False)
plt.show()


# Eşik Değeri raporu

threshold_table = pd.DataFrame({
    "Yöntem": ["Global Thresholding", "Otsu Thresholding"],
    "Eşik Değeri": [global_thresh, otsu_thresh]
})

threshold_table

print("")
print("3.1 Thresholding: Lezyon bölgesini arka plandan ayırmak için farklı eşikleme yöntemleri denenmiştir.")
print("")


#### 3.2 Binary Görüntü Üretimi ve Yöntem Seçimi 

In [ ]:
def preprocess_image(img_path):
    img = Image.open(img_path).convert("L")
    cropped = crop_image_percent(img, crop_percent=0.10)
    cropped_np = np.array(cropped)
    equalized = exposure.equalize_hist(cropped_np)
    blurred = median(equalized, disk(3))
    return blurred

NUM_SHOW = NUM_IMAGES
NUM_SHOW = min(NUM_SHOW, len(final_paths))

global_thresh = 0.5

plt.figure(figsize=(12, NUM_SHOW * 2.5))

for i in range(NUM_SHOW):
    img_path = final_paths[i]
    img_pre = preprocess_image(img_path)

    # Global Threshold
    binary_global = img_pre > global_thresh

    # Otsu Threshold
    otsu_thresh = threshold_otsu(img_pre)
    binary_otsu = img_pre > otsu_thresh

    # Global
    plt.subplot(NUM_SHOW, 3, 3*i + 1)
    plt.imshow(binary_global, cmap="gray")
    plt.axis("off")
    plt.title("Global", fontsize=9)

    # Otsu
    plt.subplot(NUM_SHOW, 3, 3*i + 2)
    plt.imshow(binary_otsu, cmap="gray")
    plt.axis("off")
    plt.title("Otsu", fontsize=9)

    # Orijinal (referans)
    plt.subplot(NUM_SHOW, 3, 3*i + 3)
    plt.imshow(img_pre, cmap="gray")
    plt.axis("off")
    plt.title("Preprocessed", fontsize=9)

plt.tight_layout()
plt.show()
print("")
print("3.2 Binary Maske: Seçilen thresholding yöntemi ile lezyonu temsil eden ikili maske elde edilmiştir.")
print("")


### 4) Post-Processing 

### 4.1 Morfolojik Operatörler

In [ ]:
def get_binary_mask(img_path):
    img = Image.open(img_path).convert("L")
    cropped = crop_image_percent(img, crop_percent=0.10)
    cropped_np = np.array(cropped)
    equalized = exposure.equalize_hist(cropped_np)
    blurred = median(equalized, disk(3))
    otsu_thresh = threshold_otsu(blurred)
    binary = blurred > otsu_thresh
    return binary


NUM_SHOW = NUM_IMAGES
NUM_SHOW = min(NUM_SHOW, len(final_paths))

selem = disk(3)

plt.figure(figsize=(10, NUM_SHOW * 2.5))

for i in range(NUM_SHOW):
    binary_mask = get_binary_mask(final_paths[i])

  
    opened = opening(binary_mask, selem)   
    morph_result = closing(opened, selem)

   
    plt.subplot(NUM_SHOW, 2, 2*i + 1)
    plt.imshow(binary_mask, cmap="gray")
    plt.axis("off")
    plt.title("Binary Mask", fontsize=9)

   
    plt.subplot(NUM_SHOW, 2, 2*i + 2)
    plt.imshow(morph_result, cmap="gray")
    plt.axis("off")
    plt.title("After Morphology", fontsize=9)

plt.tight_layout()
plt.show()
print("")
print("4.1 Morfoloji: Binary maskede oluşan küçük gürültüler temizlenmiş ve lezyon bölgesi bütünlüğü artırılmıştır.")
print("")



#### 4.2 Connected Component Labeling – CCL  

In [ ]:

def apply_ccl_and_select_largest(binary_mask):
    mask_uint8 = (binary_mask.astype(np.uint8)) * 255
    num_labels, labels = cv2.connectedComponents(mask_uint8)
    component_count = num_labels - 1
    
    areas = {}
    for label in range(1, num_labels):
        areas[label] = np.sum(labels == label)

    if len(areas) > 0:
        largest_label = max(areas, key=areas.get)
        largest_component = (labels == largest_label)
    else:
        largest_component = np.zeros_like(binary_mask, dtype=bool)

    return labels, component_count, largest_component


NUM_SHOW = min(NUM_IMAGES, len(final_paths))
plt.figure(figsize=(15, NUM_SHOW * 4))

all_component_counts = []

for i in range(NUM_SHOW):
    img_path = final_paths[i]
    
    
    original_bgr = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(original_bgr, cv2.COLOR_BGR2RGB)
    

    binary_mask = get_binary_mask(img_path) 
    

    labels, count, largest = apply_ccl_and_select_largest(binary_mask)
    all_component_counts.append(count)


    if img_rgb.shape[:2] != largest.shape:
        img_rgb = cv2.resize(img_rgb, (largest.shape[1], largest.shape[0]))

 
    plt.subplot(NUM_SHOW, 3, 3*i + 1)
    plt.imshow(labels, cmap="nipy_spectral")
    plt.axis("off")
    plt.title(f"Görüntü {i+1}: CCL Labels", fontsize=9)

  
    plt.subplot(NUM_SHOW, 3, 3*i + 2)
    plt.imshow(largest, cmap="gray")
    plt.axis("off")
    plt.title("Selected ROI (Largest)", fontsize=9)

   
    overlay = label2rgb(largest, image=img_rgb, bg_label=0, alpha=0.3)
    plt.subplot(NUM_SHOW, 3, 3*i + 3)
    plt.imshow(overlay)
    plt.axis("off")
    plt.title("RGB Overlay", fontsize=9)

plt.tight_layout()
plt.show()


distribution = Counter(all_component_counts)
print("\nBileşen Sayısı Dağılımı (Arka plan hariç):")
for k in sorted(distribution.keys()):
    durum = "(İdeal)" if k == 1 else ""
    print(f"{distribution[k]} görüntüde {k} bileşen {durum}")

print("\n4.2 CCL: Bağlı bileşen analizi tamamlandı ve Overlay görselleri oluşturuldu.")
print("")

### 5) Öznitelik (Feature) Çıkarımı 

#### 5.1 First-Order (İstatistiksel) Özellikler 

In [ ]:


def extract_first_order_features(roi_pixels):
    if roi_pixels.size == 0:
        return {"mean": 0, "std": 0, "variance": 0, "skewness": 0, "kurtosis": 0, "min": 0, "max": 0, "entropy": 0}
    
    return {
        "mean": np.mean(roi_pixels),
        "std": np.std(roi_pixels),
        "variance": np.var(roi_pixels),
        "skewness": skew(roi_pixels),
        "kurtosis": kurtosis(roi_pixels),
        "min": np.min(roi_pixels),
        "max": np.max(roi_pixels),
        "entropy": entropy(np.histogram(roi_pixels, bins=256, density=True)[0] + 1e-7)
    }

def extract_roi_pixels(gray_img, binary_mask):
    """Maskenin kapsadığı gri seviye pikselleri dizi olarak döndürür."""
    # Maske ve görüntü boyutunun aynı olduğundan emin oluyoruz
    if gray_img.shape != binary_mask.shape:
        binary_mask = cv2.resize(binary_mask.astype(np.uint8), (gray_img.shape[1], gray_img.shape[0]))
        binary_mask = binary_mask > 0
    return gray_img[binary_mask > 0]


print("\n5.1 First-Order Features: ROI içindeki piksel yoğunluklarına ait istatistiksel öznitelikler hesaplanıyor...\n")

feature_list = []

for img_path in final_paths:
 
    img_pil = Image.open(img_path).convert("L")
    
 
    cropped_gray_pil = crop_image_percent(img_pil, crop_percent=0.10)
    gray = np.array(cropped_gray_pil)

    # Binary mask üretimi ve CCL ile en büyük parçanın seçimi
    binary_mask = get_binary_mask(img_path)
    _, _, roi_mask = apply_ccl_and_select_largest(binary_mask)

    roi_pixels = extract_roi_pixels(gray, roi_mask)

    if roi_pixels.size == 0:
        print(f"Uyarı: {os.path.basename(img_path)} için ROI boş çıktı, atlanıyor.")
        continue


    features = extract_first_order_features(roi_pixels)
    

    features["image_id"] = os.path.basename(img_path)

    features["label"] = img_path.split(os.sep)[-2]

    feature_list.append(features)


first_order_df = pd.DataFrame(feature_list)



print(f"Toplam hesaplanan satır sayısı: {len(first_order_df)}")
#display(first_order_df.head())
display(first_order_df)





#### 5.2 2D Shape (Şekil) Özellikleri

In [ ]:
print("")
print("5.2 Shape Features: ROI’nin geometrik yapısını temsil eden alan, çevre ve şekil tabanlı öznitelikler çıkarılmıştır.")
print("")



def extract_shape_features(roi_mask):
    features = {}

    labeled_mask = label(roi_mask)
    regions = regionprops(labeled_mask)

    if len(regions) == 0:
        return None

    region = regions[0]  # En büyük bileşen zaten seçili


    area = region.area
    perimeter = region.perimeter

 
    circularity = (4 * np.pi * area) / (perimeter ** 2 + 1e-8)

  
    major_axis = region.major_axis_length
    minor_axis = region.minor_axis_length
    aspect_ratio = major_axis / (minor_axis + 1e-8)

 
    eccentricity = region.eccentricity
    solidity = region.solidity
    extent = region.extent


    convex_area = region.convex_area
    equivalent_diameter = region.equivalent_diameter

  
    minr, minc, maxr, maxc = region.bbox
    bbox_width = maxc - minc
    bbox_height = maxr - minr
    bbox_area = bbox_width * bbox_height

 
    features["area"] = area
    features["perimeter"] = perimeter
    features["circularity"] = circularity
    features["eccentricity"] = eccentricity
    features["solidity"] = solidity
    features["extent"] = extent
    features["major_axis_length"] = major_axis
    features["minor_axis_length"] = minor_axis
    features["aspect_ratio"] = aspect_ratio
    features["convex_area"] = convex_area
    features["equivalent_diameter"] = equivalent_diameter
    features["bbox_width"] = bbox_width
    features["bbox_height"] = bbox_height
    features["bbox_area"] = bbox_area

    return features

shape_feature_list = []

for img_path in final_paths:
    binary_mask = get_binary_mask(img_path)
    _, _, roi_mask = apply_ccl_and_select_largest(binary_mask)

    shape_features = extract_shape_features(roi_mask)

    if shape_features is None:
        continue

    shape_features["image_name"] = os.path.basename(img_path)
    shape_feature_list.append(shape_features)

shape_df = pd.DataFrame(shape_feature_list)
shape_df







#### 5.3 GLCM (Second-Order Texture) Özellikler

In [ ]:

def get_binary_mask_and_gray(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    
    _, binary_mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    return gray, binary_mask


def extract_glcm_features(gray_img, binary_mask, distances=[1, 2], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=16):
   


    glcm_input = (gray_img // (256 // levels)).astype(np.uint8)
    

    glcm_input[~binary_mask] = 0 
    

    glcm = graycomatrix(glcm_input, distances=distances, angles=angles, 
                        levels=levels, symmetric=True, normed=True)
    
 
    features = {
        "contrast": graycoprops(glcm, 'contrast').mean(),
        "dissimilarity": graycoprops(glcm, 'dissimilarity').mean(),
        "homogeneity": graycoprops(glcm, 'homogeneity').mean(),
        "energy": graycoprops(glcm, 'energy').mean(),
        "correlation": graycoprops(glcm, 'correlation').mean(),
        "ASM": graycoprops(glcm, 'ASM').mean()  # <-- Hoca bunu özellikle istiyor
    }
    
    return features


print("\n5.3 GLCM Features: ROI içindeki doku yapısı (texture) hesaplanıyor...")
print(f"Parametreler: Distances=[1, 2], Levels={16}, Angles=[0, 45, 90, 135 derece]\n")

glcm_feature_list = []

for img_path in final_paths:

    gray, binary_mask = get_binary_mask_and_gray(img_path)
    

    _, _, roi_mask = apply_ccl_and_select_largest(binary_mask)

    if roi_mask.sum() == 0:
        continue


    glcm_features = extract_glcm_features(
        gray, 
        roi_mask, 
        distances=[1, 2], 
        angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], 
        levels=16 # Gri seviye indirgeme parametresi
    )

    glcm_features["image_name"] = os.path.basename(img_path)
    glcm_feature_list.append(glcm_features)


glcm_df = pd.DataFrame(glcm_feature_list)
#display(glcm_df.head())
display(glcm_df)




#### 5.4 Feature Tablosu ve Çıktıların Yorumlanması 

In [ ]:
print("\n5.4 Feature Tablosu ve Çıktıların Yorumlanması")
print("-" * 50)


try:

    if 'features_df' in locals():
        f_df = features_df.copy()
    elif 'first_order_df' in locals():
        f_df = first_order_df.copy()
    else:
        raise NameError("5.1 (First-order) tablosu bulunamadı!")

  
    if 'shape_df' in locals():
        s_df = shape_df.copy()
    else:
        raise NameError("5.2 (Shape) tablosu bulunamadı!")

 
    if 'glcm_df' in locals():
        g_df = glcm_df.copy()
    else:
        raise NameError("5.3 (GLCM) tablosu bulunamadı!")

 
  
    for d in [f_df, s_df, g_df]:
        if "image_name" in d.columns:
            d.rename(columns={"image_name": "image_id"}, inplace=True)
        if "roi_id" not in d.columns:
            d["roi_id"] = 0

  
    df_final = f_df.merge(s_df, on=["image_id", "roi_id"], how="inner")
    df_final = df_final.merge(g_df, on=["image_id", "roi_id"], how="inner")


    if "label" not in df_final.columns:
        # Resim adının ilk kısmını (örn: nv_001 -> nv) label olarak alır
        df_final["label"] = df_final["image_id"].apply(lambda x: x.split("_")[0] if "_" in x else "lezyon")

   
    csv_path = "feature_table.csv"
    df_final.to_csv(csv_path, index=False)

 
    total_images = df_final["image_id"].nunique()
    total_rois = len(df_final)
 
    feature_cols = [c for c in df_final.columns if c not in ["image_id", "roi_id", "label"]]
    
    roi_counts = df_final.groupby("image_id").size()
    sapma_sayisi = sum(roi_counts > 1)

    print(f"1. Toplam İşlenen Görüntü Sayısı : {total_images}")
    print(f"2. Toplam Üretilen ROI Sayısı    : {total_rois}")
    print(f"3. Öznitelik Sayısı (Sütunlar)   : {len(feature_cols)}")
    print(f"4. Birden Fazla ROI Olan Görüntü: {sapma_sayisi}")
    
   
    if sapma_sayisi > 0:
        print(f"NOT: {sapma_sayisi} görüntüde birden fazla ROI oluşmuştur.")
        print("NEDEN: Segmentasyon sırasında lezyonun parçalı görünmesi veya küçük gürültülerin temizlenememesidir.")
    else:
        print("NOT: Hedeflenen '1 Görüntü = 1 ROI' yapısı başarıyla sağlanmıştır.")

    print(f"\nDosya kaydedildi: {csv_path}")
    #display(df_final.head())
    display(df_final)

except NameError as e:
    print(f"HATA: {e}")
    print("İPUCU: Lütfen 5.1, 5.2 ve 5.3 hücrelerini yukarıdan aşağıya sırayla çalıştırdığınızdan emin olun.")
except Exception as e:
    print(f"Beklenmedik bir hata oluştu: {e}")